# DOUBLE ENTRY CONFIRMATION

This script compares the two independent entries for consistency (by Jacob Wenegrat & Emma Bonanno vs Julia Wenegrat).

In [ ]:
%%capture
!apt-get install libproj-dev proj-data proj-bin
!apt-get install libgeos-dev
!pip install geoplot
!pip install gsw
!pip install xarray
!pip install netcdf4
!apt-get install texlive texlive-xetex texlive-latex-extra pandoc
!pip install pypandoc

In [ ]:
from google.colab import auth
auth.authenticate_user()
import gspread
from oauth2client.client import GoogleCredentials
gc = gspread.authorize(GoogleCredentials.get_application_default())

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import gsw
import pandas as pd
import pyproj as proj
import scipy.interpolate as interpolate
from IPython.display import clear_output
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

## LOAD EMMA VERSION

In [ ]:
url_name='https://docs.google.com/spreadsheets/d/1VJWvSL5mzGKGU3bgp6hEPqDOPDhL4oRiXVur2YMxWQQ/edit#gid=1011727121'
spreadsheet=gc.open_by_url(url_name)

def getSheetAsXarray(sheetname):
  prof_sheet = spreadsheet.worksheet(sheetname)
  prof_data = prof_sheet.get_all_values()
  df = pd.DataFrame(prof_data)
  df.columns = df.iloc[0]
  df = df.iloc[1:]
  df = df.astype({'Temperature':'float64', 
                  'Lat degrees':'float64', 
                  'Lat minutes':'float64', 
                  'Lon degrees':'float64',
                  'Lon minutes':'float64',
                  'Depth':'float64',
                  'Station':'str'
                  })
  #df = df.set_index(['Station'])
  ds = df.to_xarray()
  ds['lat'] = ds['Lat degrees'] + ds['Lat minutes']/60
  ds['lon'] = ds['Lon degrees'] + ds['Lon minutes']/60
  return ds


ds = getSheetAsXarray('Parsed Profile Data')

#stationkeys = np.delete( stationkeys, 11)
dfv = ds.to_dataframe() 
dfv = dfv.set_index('Station')
dfv = dfv.drop('3') # Stations 3 and 6 are in not in IO
dfv = dfv.drop('6')
dfv = dfv.drop('200-206')
dfv = dfv.drop('135/137')
dfv = dfv.drop('149/152/153')
dfv = dfv.reset_index()
dfv['cruise'] = 'Valdivia'
stationkeys = np.unique(dfv.Station.values)

#dsvi = getSheetAsXarray('Valdivia Interpolated')
#stationkeysvi = np.unique(dsvi.Station.values)

dsp = getSheetAsXarray('Planet Data')

dfp = dsp.to_dataframe() 
dfp['cruise'] = 'Planet'
stationkeysplanet = np.unique(dfp.Station.values)

dsgaz = getSheetAsXarray('Gazelle')

dfg = dsgaz.to_dataframe() 
dfg['cruise'] = 'Gazelle'
stationkeysgazelle = np.unique(dfg.Station.values)

ALL_CRUISES = pd.concat([dfg, dfv, dfp])
ALL_CRUISES = ALL_CRUISES.reset_index(drop=True)
ALL_CRUISES.Date = pd.to_datetime(ALL_CRUISES.Date)
ALL_CRUISES['Month'] = pd.DatetimeIndex(ALL_CRUISES['Date']).month

DF_EMMA = ALL_CRUISES.copy()
#DF_EMMA

### JULIA VERSION

In [ ]:
url_name='https://docs.google.com/spreadsheets/d/1VJWvSL5mzGKGU3bgp6hEPqDOPDhL4oRiXVur2YMxWQQ/edit#gid=1011727121'
url_name='https://docs.google.com/spreadsheets/d/10lhSgKMsG4K8IxhVCH6r36rPH8V219s0fq7D3S5K03w'
spreadsheet=gc.open_by_url(url_name)

def getSheetAsXarray(sheetname):
  prof_sheet = spreadsheet.worksheet(sheetname)
  prof_data = prof_sheet.get_all_values()
  df = pd.DataFrame(prof_data)
  df.columns = df.iloc[0]
  df = df.iloc[1:]
  df = df.astype({'Temperature':'float64', 
                  'Lat degrees':'float64', 
                  'Lat minutes':'float64', 
                  'Lon degrees':'float64',
                  'Lon minutes':'float64',
                  'Depth':'float64',
                  'Station':'str'
                  })
  #df = df.set_index(['Station'])
  ds = df.to_xarray()
  ds['lat'] = ds['Lat degrees'] + ds['Lat minutes']/60
  ds['lon'] = ds['Lon degrees'] + ds['Lon minutes']/60
  return ds

ds = getSheetAsXarray('Valdivia')

#stationkeys = np.delete( stationkeys, 11)
dfv = ds.to_dataframe() 
dfv = dfv.set_index('Station')
dfv = dfv.reset_index()
dfv['cruise'] = 'Valdivia'
stationkeys = np.unique(dfv.Station.values)

#dsvi = getSheetAsXarray('Valdivia Interpolated')
#stationkeysvi = np.unique(dsvi.Station.values)

dsp = getSheetAsXarray('Planet')

dfp = dsp.to_dataframe() 
dfp['cruise'] = 'Planet'
stationkeysplanet = np.unique(dfp.Station.values)

dsgaz = getSheetAsXarray('Gazelle')

dfg = dsgaz.to_dataframe() 
dfg['cruise'] = 'Gazelle'
stationkeysgazelle = np.unique(dfg.Station.values)

ALL_CRUISES = pd.concat([dfg, dfv, dfp])
ALL_CRUISES = ALL_CRUISES.reset_index(drop=True)
ALL_CRUISES.Date = pd.to_datetime(ALL_CRUISES.Date)
ALL_CRUISES['Month'] = pd.DatetimeIndex(ALL_CRUISES['Date']).month

DF_JULIA = ALL_CRUISES.copy()
#DF_JULIA

### SUBSET APPROACH

Go through every row in Emma's and try to find a counterpart

In [ ]:
IGNORE_LIST = [
               396, #Valdivia with comment (shifted station)
               398, #Valdivia with comment (shifted station)
               427, #Valdivia with comment (shifted station)
               428,
               429, 
               430,
               432, #EMMA is correct (hard to find data point)
               460, #EMMA is correct (hard to find data point)
               472, #EMMA is correct (hard to find data point)
]

In [ ]:
counter = 0
for index, row in DF_EMMA.iterrows():
  row_j = DF_JULIA[(DF_JULIA.Station == row.Station)]
  row_j = row_j[row_j.cruise == row.cruise]
  row_j = row_j[row_j.Depth == row.Depth]
  if row_j.index.size == 0:
    if (index not in IGNORE_LIST):
      if row.Depth != np.float64(row.Bottom):
        print(f'No match for index {index}')
    #print(f'Station: {row.Station}, Depth: {row.Depth}, Cruise: {row.cruise}')
        counter += 1
  else: # Rows match
    if row_j.Date.values != row.Date:
      print(f'Error in date for index: {index}')
    if row_j['Lon degrees'].values != row['Lon degrees']:
      print(f'Error in Lon Degrees for index: {index}')
    if row_j['Lon minutes'].values != row['Lon minutes']:
      print(f'Error in Lon Minutes for index: {index}')
    if row_j['Lat degrees'].values != row['Lat degrees']:
      print(f'Error in Lat Degrees for index: {index}')
    if row_j['Temperature'].values != row['Temperature']:
      print(f'Error in Temperature for index: {index}')
print(f'Number of missing rows: {counter}')


Number of missing rows: 0


In [ ]:
# Track down missing data in Julia's
index = 396
var = 'Temperature'
row = DF_EMMA.iloc[index]
print(row)
row_j = DF_JULIA[(DF_JULIA.Station == row.Station)]
row_j = row_j[row_j.cruise == row.cruise]
row_j = row_j[row_j.Depth == row.Depth]
st = row_j['Station']
print(row_j)
print(row_j[var])


Station                                    190
Date                       1899-01-30 00:00:00
Lat degrees                                 -3
Lat minutes                                -22
Lon degrees                                101
Lon minutes                                 12
Bottom                                    1280
Depth                                      903
Temperature                                  6
Notes          Station 186 (DUPLICATE, REMOVE)
lat                                   -3.36667
lon                                      101.2
cruise                                Valdivia
                                              
Month                                        1
Name: 396, dtype: object
Empty DataFrame
Columns: [Station, Date, Lat degrees, Lat minutes, Lon degrees, Lon minutes, Depth, Temperature, Bottom Depth, Notes, lat, lon, cruise, Month]
Index: []
Series([], Name: Temperature, dtype: float64)


In [ ]:
IGNORE_LIST_J = list(range(294, 347)) #These are in the Indonesian seas (not used)
TEMP_LIST = [
             484, # Parsed in EMMA is correct
             496, 
             ]

IGNORE_LIST_J = IGNORE_LIST_J + TEMP_LIST

In [ ]:
# Go the other way
counter = 0
for index, row in DF_JULIA.iterrows():
  row_j = DF_EMMA[(DF_EMMA.Station == row.Station)]
  row_j = row_j[row_j.cruise == row.cruise]
  row_j = row_j[row_j.Depth == row.Depth]
  if row_j.index.size == 0:
    if (index not in IGNORE_LIST_J):
      #if row.Depth != np.float64(row.Bottom):
        print(f'No match for index {index}')
    #print(f'Station: {row.Station}, Depth: {row.Depth}, Cruise: {row.cruise}')
        counter += 1
  else: # Rows match
    if row_j.Date.values != row.Date:
      print(f'Error in date for index: {index}')
    if row_j['Lon degrees'].values != row['Lon degrees']:
      print(f'Error in Lon Degrees for index: {index}')
    if row_j['Lon minutes'].values != row['Lon minutes']:
      print(f'Error in Lon Minutes for index: {index}')
    if row_j['Lat degrees'].values != row['Lat degrees']:
      print(f'Error in Lat Degrees for index: {index}')
    if row_j['Temperature'].values != row['Temperature']:
      print(f'Error in Temperature for index: {index}')
print(f'Number of missing rows: {counter}')

Number of missing rows: 0


In [ ]:
# track down missing data in Emma's
index = 517
var = 'Temperature'
row = DF_JULIA.iloc[index]
print(row)
row_j = DF_EMMA[(DF_EMMA.Station == row.Station)]
row_j = row_j[row_j.cruise == row.cruise]
row_j = row_j[row_j.Depth == row.Depth]
st = row_j['Station']
print(row_j)
print(row_j[var])



Station                         268
Date            1899-04-01 00:00:00
Lat degrees                       9
Lat minutes                       6
Lon degrees                      53
Lon minutes                      41
Depth                             0
Temperature                    27.5
Bottom Depth                   5064
Notes                              
lat                             9.1
lon                         53.6833
cruise                     Valdivia
Month                             4
Name: 517, dtype: object
Empty DataFrame
Columns: [Station, Date, Lat degrees, Lat minutes, Lon degrees, Lon minutes, Bottom, Depth, Temperature, Notes, lat, lon, cruise, , Month]
Index: []
Series([], Name: Temperature, dtype: float64)


### ITERATIVE APPROACH

Note sure this works well, leads to lots of flags.

In [ ]:
for index, row in DF_JULIA.iterrows():
  # Make sure the stations are setup the same
  if DF_EMMA.loc[index, 'Station'] != row['Station']:
    print(f'Stations do not match at index: {index}')
  else:
    # Check dates are the same
    if DF_EMMA.loc[index, 'Date'] != row['Date']:
      print(f'Dates do not match at index: {index}')
   # Check Lat/Lon are the same
    if (DF_EMMA.loc[index, 'lat'] != row['lat']) | (DF_EMMA.loc[index, 'lon'] != row['lon']):
      print(f'Lat/Lon do not match at index: {index}')
  # Check depth is the same
    if (DF_EMMA.loc[index, 'Depth'] != row['Depth']):
      print(f'Depth does not match at index: {index}')
  # Check Temperature is the same
    if (DF_EMMA.loc[index, 'Temperature'] != row['Temperature']):
      print(f'Temperature does not match at index: {index}')



Stations do not match at index: 71
Depth does not match at index: 72
Temperature does not match at index: 72
Depth does not match at index: 73
Temperature does not match at index: 73
Stations do not match at index: 74
Stations do not match at index: 75
Depth does not match at index: 76
Temperature does not match at index: 76
Depth does not match at index: 77
Temperature does not match at index: 77
Depth does not match at index: 78
Temperature does not match at index: 78
Depth does not match at index: 79
Temperature does not match at index: 79
Depth does not match at index: 80
Temperature does not match at index: 80
Stations do not match at index: 81
Stations do not match at index: 82
Depth does not match at index: 83
Temperature does not match at index: 83
Depth does not match at index: 84
Temperature does not match at index: 84
Depth does not match at index: 85
Temperature does not match at index: 85
Depth does not match at index: 86
Temperature does not match at index: 86
Depth does 

KeyError: ignored

In [ ]:
DF_EMMA.iloc[2]

Station                         38
Date           1874-10-04 00:00:00
Lat degrees                    -34
Lat minutes                   -6.5
Lon degrees                     18
Lon minutes                    6.5
Bottom                         214
Depth                           91
Temperature                    9.7
Notes                             
lat                       -34.1083
lon                        18.1083
cruise                     Gazelle
                               NaN
Month                           10
Name: 2, dtype: object

In [ ]:
DF_JULIA.iloc[0:10]

,Station,Date,Lat degrees,Lat minutes,Lon degrees,Lon minutes,Depth,Temperature,Bottom Depth,Notes,lat,lon,cruise,Month
0,38,1874-10-04,-34.0,-6.5,18.0,6.5,0.0,14.8,214,,-34.108333,18.108333,Gazelle,10
1,38,1874-10-04,-34.0,-6.5,18.0,6.5,91.0,11.6,214,in 55 meters (in note). Also both numbers in t...,-34.108333,18.108333,Gazelle,10
2,38,1874-10-04,-34.0,-6.5,18.0,6.5,146.0,8.1,214,,-34.108333,18.108333,Gazelle,10
3,39,1874-10-06,-35.0,-23.0,16.0,30.5,0.0,15.4,,,-35.383333,16.508333,Gazelle,10
4,39,1874-10-06,-35.0,-23.0,16.0,30.5,91.0,14.4,,,-35.383333,16.508333,Gazelle,10
5,39,1874-10-06,-35.0,-23.0,16.0,30.5,183.0,13.3,,,-35.383333,16.508333,Gazelle,10
6,39,1874-10-06,-35.0,-23.0,16.0,30.5,366.0,8.6,,,-35.383333,16.508333,Gazelle,10
7,39,1874-10-06,-35.0,-23.0,16.0,30.5,549.0,5.4,,,-35.383333,16.508333,Gazelle,10
8,40,1874-10-08,-39.0,-9.5,20.0,56.0,0.0,19.2,,,-39.158333,20.933333,Gazelle,10
9,40,1874-10-08,-39.0,-9.5,20.0,56.0,91.0,18.3,,in 55 meters (in note). Also both numbers in t...,-39.158333,20.933333,Gazelle,10
